In [12]:
import torch 
import numpy as np
import pylab as plt
from matplotlib import pyplot
from matplotlib.pyplot import figure
from os.path import dirname, realpath
import pathlib
import sv_utils
from singleNeuron import preSpikes, nextSpikes, STDPLIFDensePopulation, convergence_level, get_metrics, get_metrics_long_pat

# Resultados de las corridas Masivas - Arquitectura Paralela
La arquitectura paralela esta compuesta por un tren sin delay y por otro tren que tiene delay, al final hay una neurona que integra ambas salidas

In [20]:
for exp in ['/15_0_0', '/15_5_0', '/0_0_15']:
    for sample in ['p_18', 'p_18_5', 'p_18_20']:
        for run in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]:
            for th in [3.0, 3.5, 4.0]:
                for inc in [0.25, 0.5, 1.0]:
                    #---------- Load snn spikes ------------#
                    Sin_out_seq = torch.load('./cleanDSNN/results/old_results/massive_runs/18/0_0_0/' + sample + '/training/'+ str(run) +'/layer_5_conv.pt')
                    Sin_out_delay = torch.load('./cleanDSNN/results/old_results/massive_runs/18' + exp + '/' + sample + '/training/'+ str(run) +'/layer_5_conv.pt')
                    combined_out = torch.cat((Sin_out_seq, Sin_out_delay), 1)

                    #--------- Train Single Neuron ------------#
                    Sin = combined_out.clone().detach()
                    T = Sin.shape[0]
                    N_out = 1
                    N_in = Sin.shape[1]
                    th = th

                    singleNeuron = STDPLIFDensePopulation(in_channels=N_in, out_channels=N_out,
                                                            weight=0.70, alpha=float(np.exp(-1e-3/10e-3)),
                                                            beta=float(np.exp(-1e-3/2e-5)), delay=0,
                                                            th=th, a_plus=.009125, a_minus=.0125625,
                                                            w_max=1.)


                    # Pre-procesamos PSpikes y NSpikes
                    dt_ltp = 20  # Cantidad de timesteps que miro hacia atras
                    dt_ltd = 70  # Cantidad de timesteps que miro hacia delante
                    PSpikes = preSpikes(T, dt_ltp, torch.zeros(T, N_in), Sin)
                    NSpikes = nextSpikes(T, dt_ltd, torch.zeros(T, N_in), Sin)

                    # Realizamos el entrenamiento STDP
                    Uprobe = np.empty([T, N_out])
                    Iprobe = np.empty([T, N_out])
                    Sprobe = np.empty([T, N_out])
    
                    n = 0
                    t = 0
                    c_l = 1.
                    while (c_l > 0.01) and (t < 300000):
                        state = singleNeuron.forward(Sin[n].unsqueeze(0), PSpikes[n], NSpikes[n-1])
                        Uprobe[n] = state.U.data.numpy()
                        Iprobe[n] = state.I.data.numpy()
                        Sprobe[n] = state.S.data.numpy()
                        n += 1
                        if n >= T:
                            n = 0
                        if (t == 220000):
                            singleNeuron.th = singleNeuron.th + inc
                        c_l = convergence_level(singleNeuron.fc_layer.weight[0])
                        t += 1


                    #----------- Test Single Neuron Output -------------#
                    Sin_out_seq = torch.load('./cleanDSNN/results/old_results/massive_runs/18/0_0_0/' + sample + '/testing/'+ str(run) +'/layer_5_conv.pt')
                    Sin_out_delay = torch.load('./cleanDSNN/results/old_results/massive_runs/18' + exp + '/' + sample + '/testing/'+ str(run) +'/layer_5_conv.pt')
                    combined_out = torch.cat((Sin_out_seq, Sin_out_delay), 1)
            
                    pat_times = np.load('./patterns/sequences/massive_runs/18/' + sample + '/testing/'+ str(run) +'/pat_test_' + sample + '.npy')
                    pat_number = np.load('./patterns/sequences/massive_runs/18/' + sample + '/testing/'+ str(run) +'/data_test_' + sample + '.npy')
                        
                    Sin = combined_out.clone().detach()
                    T = Sin.shape[0]
                    N_out = 1
                    N_in = Sin.shape[1]
                
                    #singleNeuron.th = singleNeuron.th + 0.25

                    # Realizamos el entrenamiento STDP
                    Uprobe = np.empty([T, N_out])
                    Iprobe = np.empty([T, N_out])
                    Sprobe = np.empty([T, N_out])
                    for n in range(T):
                        state = singleNeuron.forward_no_learning(Sin[n].unsqueeze(0))
                        Uprobe[n] = state.U.data.numpy()
                        Iprobe[n] = state.I.data.numpy()
                        Sprobe[n] = state.S.data.numpy()

                    print('Exp: ' + exp + ' Sample: '+ sample + ' Run: '  + str(run) + ' th: ' + str(th) )
                    accuracy, precision, recall, f1, fake_alarms, missed_alarms, fake_spikes_numbers = get_metrics_long_pat(T - 15, pat_times, Sprobe, 30, pat_number)
                    print("accuracy: " + str(accuracy))
                    print("precision: " + str(precision))
                    print("recall: " + str(recall))
                    print("f1: " + str(f1))
                    print("fake_alarms: " + str(fake_alarms))
                    print("missed_alarms: " + str(missed_alarms))
                    print('  ')
            
                    #----------- Save Results -----------#
                    str_th = str(th) + str(th + inc)
                    str_th = str_th[0] + '_' + str_th[2] + '-' + str_th[3] + '_' + str_th[5]
                    location = './cleanDSNN/results/old_results/massive_runs/18' + exp + '/' + sample + '/metrics/'+ str(run) + '/'
                    metrics = np.array([accuracy, precision, recall, f1, fake_alarms, missed_alarms], dtype=float)
                    pathlib.Path(location).mkdir(parents=True, exist_ok=True)
                    np.save(location + 'test_metrics_' + str_th + '.npy', metrics)
            
                    #----------- Test Dif Sample Single Neuron Output -------------#
                    Sin_out_seq = torch.load('./cleanDSNN/results/old_results/massive_runs/18/0_0_0/' + sample + '/dif_sample/'+ str(run) +'/layer_5_conv.pt')
                    Sin_out_delay = torch.load('./cleanDSNN/results/old_results/massive_runs/18' + exp + '/' + sample + '/dif_sample/'+ str(run) +'/layer_5_conv.pt')
                    combined_out = torch.cat((Sin_out_seq, Sin_out_delay), 1)
            
                    pat_times = np.load('./patterns/sequences/massive_runs/18/p_18_dif_sample/pat_p_18_dif_sample.npy')
                    pat_number = np.load('./patterns/sequences/massive_runs/18/p_18_dif_sample/data_p_18_dif_sample.npy')
                        
                    Sin = combined_out.clone().detach()
                    T = Sin.shape[0]
                    N_out = 1
                    N_in = Sin.shape[1]
                    

                    # Realizamos el entrenamiento STDP
                    Uprobe = np.empty([T, N_out])
                    Iprobe = np.empty([T, N_out])
                    Sprobe = np.empty([T, N_out])
                    for n in range(T):
                        state = singleNeuron.forward_no_learning(Sin[n].unsqueeze(0))
                        Uprobe[n] = state.U.data.numpy()
                        Iprobe[n] = state.I.data.numpy()
                        Sprobe[n] = state.S.data.numpy()

                    print('(Dif Sample) Exp: ' + exp + ' Sample: '+ sample + ' Run: '  + str(run) + ' th: ' + str(th) )
                    accuracy, precision, recall, f1, fake_alarms, missed_alarms, fake_spikes_numbers = get_metrics_long_pat(T - 15, pat_times, Sprobe, 30, pat_number)
                    print("accuracy: " + str(accuracy))
                    print("precision: " + str(precision))
                    print("recall: " + str(recall))
                    print("f1: " + str(f1))
                    print("fake_alarms: " + str(fake_alarms))
                    print("missed_alarms: " + str(missed_alarms))
                    print('  ')
        
                    #----------- Save Results -----------#
                    location = './cleanDSNN/results/old_results/massive_runs/18' + exp + '/' + sample + '/metrics/'+ str(run) + '/'
                    metrics = np.array([accuracy, precision, recall, f1, fake_alarms, missed_alarms], dtype=float)
                    pathlib.Path(location).mkdir(parents=True, exist_ok=True)
                    np.save(location + 'dif_sample_metrics_' + str_th + '.npy', metrics)
                

Exp: /15_0_0 Sample: p_18 Run: 0 th: 3.0
accuracy: 0.7485988791032826
precision: 0.3812807881773399
recall: 1.0
f1: 0.5520684736091298
fake_alarms: 0.2974893415442918
missed_alarms: 0.0
  
(Dif Sample) Exp: /15_0_0 Sample: p_18 Run: 0 th: 3.0
accuracy: 0.6547427652733119
precision: 0.24509803921568626
recall: 0.5754475703324808
f1: 0.3437738731856379
fake_alarms: 0.33047210300429186
missed_alarms: 0.42455242966751916
  
Exp: /15_0_0 Sample: p_18 Run: 0 th: 3.0
accuracy: 0.7754203362690152
precision: 0.40822784810126583
recall: 1.0
f1: 0.5797752808988764
fake_alarms: 0.2657508289909995
missed_alarms: 0.0
  
(Dif Sample) Exp: /15_0_0 Sample: p_18 Run: 0 th: 3.0
accuracy: 0.6748392282958199
precision: 0.24635922330097088
recall: 0.5191815856777494
f1: 0.33415637860082303
fake_alarms: 0.296137339055794
missed_alarms: 0.48081841432225064
  
Exp: /15_0_0 Sample: p_18 Run: 0 th: 3.0
accuracy: 0.9987990392313851
precision: 0.9948453608247423
recall: 0.9974160206718347
f1: 0.9961290322580645
fa

(Dif Sample) Exp: /15_0_0 Sample: p_18 Run: 2 th: 3.0
accuracy: 0.5655144694533762
precision: 0.23214285714285715
recall: 0.7647058823529411
f1: 0.3561643835616438
fake_alarms: 0.4716261325703386
missed_alarms: 0.23529411764705882
  
Exp: /15_0_0 Sample: p_18 Run: 2 th: 3.0
accuracy: 0.7409927942353883
precision: 0.3766859344894027
recall: 1.0
f1: 0.5472358292512246
fake_alarms: 0.3070716658756526
missed_alarms: 0.0
  
(Dif Sample) Exp: /15_0_0 Sample: p_18 Run: 2 th: 3.0
accuracy: 0.637459807073955
precision: 0.24321608040201004
recall: 0.618925831202046
f1: 0.3492063492063492
fake_alarms: 0.3590844062947067
missed_alarms: 0.38107416879795397
  
Exp: /15_0_0 Sample: p_18 Run: 2 th: 3.5
accuracy: 0.6761409127301842
precision: 0.3258333333333333
recall: 1.0
f1: 0.491514770584538
fake_alarms: 0.38395823445657334
missed_alarms: 0.0
  
(Dif Sample) Exp: /15_0_0 Sample: p_18 Run: 2 th: 3.5
accuracy: 0.592443729903537
precision: 0.23261802575107296
recall: 0.6930946291560103
f1: 0.3483290488

Exp: /15_0_0 Sample: p_18 Run: 4 th: 3.5
accuracy: 0.9423538831064852
precision: 0.7225433526011561
recall: 1.0
f1: 0.8389261744966443
fake_alarms: 0.06782854451248234
missed_alarms: 0.0
  
(Dif Sample) Exp: /15_0_0 Sample: p_18 Run: 4 th: 3.5
accuracy: 0.8018488745980707
precision: 0.27232142857142855
recall: 0.15601023017902813
f1: 0.1983739837398374
fake_alarms: 0.07773009060562709
missed_alarms: 0.8439897698209718
  
Exp: /15_0_0 Sample: p_18 Run: 4 th: 3.5
accuracy: 0.9943955164131305
precision: 0.9712793733681462
recall: 0.992
f1: 0.9815303430079155
fake_alarms: 0.0051813471502590676
missed_alarms: 0.008
  
(Dif Sample) Exp: /15_0_0 Sample: p_18 Run: 4 th: 3.5
accuracy: 0.8392282958199357
precision: 0.2
recall: 0.0076726342710997444
f1: 0.014778325123152709
fake_alarms: 0.005722460658082976
missed_alarms: 0.9923273657289002
  
Exp: /15_0_0 Sample: p_18 Run: 4 th: 3.5
accuracy: 0.9763811048839072
precision: 0.8640552995391705
recall: 1.0
f1: 0.927070457354759
fake_alarms: 0.027790

(Dif Sample) Exp: /15_0_0 Sample: p_18 Run: 6 th: 3.5
accuracy: 0.647508038585209
precision: 0.2525458248472505
recall: 0.6342710997442456
f1: 0.36125273124544793
fake_alarms: 0.35002384358607536
missed_alarms: 0.3657289002557545
  
Exp: /15_0_0 Sample: p_18 Run: 6 th: 3.5
accuracy: 0.7594075260208166
precision: 0.3929292929292929
recall: 1.0
f1: 0.5641769398114576
fake_alarms: 0.2849691797060218
missed_alarms: 0.0
  
(Dif Sample) Exp: /15_0_0 Sample: p_18 Run: 6 th: 3.5
accuracy: 0.6872990353697749
precision: 0.2660217654171705
recall: 0.5626598465473146
f1: 0.361247947454844
fake_alarms: 0.28946113495469716
missed_alarms: 0.4373401534526854
  
Exp: /15_0_0 Sample: p_18 Run: 6 th: 4.0
accuracy: 0.7121697357886309
precision: 0.3510830324909747
recall: 1.0
f1: 0.5197060788243153
fake_alarms: 0.3409198672356567
missed_alarms: 0.0
  
(Dif Sample) Exp: /15_0_0 Sample: p_18 Run: 6 th: 4.0
accuracy: 0.6515273311897106
precision: 0.2536231884057971
recall: 0.6265984654731458
f1: 0.36109064112

(Dif Sample) Exp: /15_0_0 Sample: p_18 Run: 8 th: 3.5
accuracy: 0.6909163987138264
precision: 0.24390243902439024
recall: 0.46035805626598464
f1: 0.31886625332152346
fake_alarms: 0.26609442060085836
missed_alarms: 0.5396419437340153
  
Exp: /15_0_0 Sample: p_18 Run: 8 th: 4.0
accuracy: 0.6533226581265013
precision: 0.27299128751210067
recall: 0.7103274559193955
f1: 0.3944055944055944
fake_alarms: 0.3574488338886245
missed_alarms: 0.28967254408060455
  
(Dif Sample) Exp: /15_0_0 Sample: p_18 Run: 8 th: 4.0
accuracy: 0.6567524115755627
precision: 0.25502645502645505
recall: 0.6163682864450127
f1: 0.36077844311377244
fake_alarms: 0.3357176919408679
missed_alarms: 0.3836317135549872
  
Exp: /15_0_0 Sample: p_18 Run: 8 th: 4.0
accuracy: 0.6537229783827062
precision: 0.2607361963190184
recall: 0.6423173803526449
f1: 0.3709090909090909
fake_alarms: 0.3441218467396478
missed_alarms: 0.35768261964735515
  
(Dif Sample) Exp: /15_0_0 Sample: p_18 Run: 8 th: 4.0
accuracy: 0.6595659163987139
precis

...

(Dif Sample) Exp: /0_0_15 Sample: p_18_20 Run: 0 th: 4.0
accuracy: 0.49115755627009644
precision: 0.22432262129804661
recall: 0.9104859335038363
f1: 0.35995955510616784
fake_alarms: 0.587029089175012
missed_alarms: 0.08951406649616368
  
Exp: /0_0_15 Sample: p_18_20 Run: 0 th: 4.0
accuracy: 0.5260208166533227
precision: 0.2347883597883598
recall: 0.9293193717277487
f1: 0.37486800422386485
fake_alarms: 0.5467863894139886
missed_alarms: 0.07068062827225131
  
(Dif Sample) Exp: /0_0_15 Sample: p_18_20 Run: 0 th: 4.0
accuracy: 0.5080385852090032
precision: 0.2272429600523903
recall: 0.887468030690537
f1: 0.3618352450469239
fake_alarms: 0.5627086313781593
missed_alarms: 0.11253196930946291
  
Exp: /0_0_15 Sample: p_18_20 Run: 0 th: 4.0
accuracy: 0.5996797437950361
precision: 0.2555379746835443
recall: 0.8455497382198953
f1: 0.39246658566221143
fake_alarms: 0.4447069943289225
missed_alarms: 0.1544502617801047
  
(Dif Sample) Exp: /0_0_15 Sample: p_18_20 Run: 0 th: 4.0
accuracy: 0.58400321543

(Dif Sample) Exp: /0_0_15 Sample: p_18_20 Run: 2 th: 4.0
accuracy: 0.5112540192926045
precision: 0.22334004024144868
recall: 0.8516624040920716
f1: 0.3538788522848034
fake_alarms: 0.5522174535050072
missed_alarms: 0.1483375959079284
  
Exp: /0_0_15 Sample: p_18_20 Run: 2 th: 4.0
accuracy: 0.5132105684547638
precision: 0.21138211382113822
recall: 0.8571428571428571
f1: 0.3391304347826087
fake_alarms: 0.5454545454545454
missed_alarms: 0.14285714285714285
  
(Dif Sample) Exp: /0_0_15 Sample: p_18_20 Run: 2 th: 4.0
accuracy: 0.530144694533762
precision: 0.22644163150492264
recall: 0.8235294117647058
f1: 0.3552123552123552
fake_alarms: 0.5245588936576061
missed_alarms: 0.17647058823529413
  
Exp: /0_0_15 Sample: p_18_20 Run: 2 th: 4.0
accuracy: 0.5844675740592474
precision: 0.22646103896103897
recall: 0.7664835164835165
f1: 0.34962406015037595
fake_alarms: 0.4465791940018744
missed_alarms: 0.23351648351648352
  
(Dif Sample) Exp: /0_0_15 Sample: p_18_20 Run: 2 th: 4.0
accuracy: 0.6004823151

(Dif Sample) Exp: /0_0_15 Sample: p_18_20 Run: 4 th: 4.0
accuracy: 0.4569935691318328
precision: 0.2132616487455197
recall: 0.9130434782608695
f1: 0.34576271186440677
fake_alarms: 0.6280400572246065
missed_alarms: 0.08695652173913043
  
Exp: /0_0_15 Sample: p_18_20 Run: 4 th: 4.0
accuracy: 0.43955164131305047
precision: 0.1986989946777055
recall: 0.8818897637795275
f1: 0.32432432432432434
fake_alarms: 0.6400566839867737
missed_alarms: 0.11811023622047244
  
(Dif Sample) Exp: /0_0_15 Sample: p_18_20 Run: 4 th: 4.0
accuracy: 0.47387459807073956
precision: 0.21736453201970443
recall: 0.9028132992327366
f1: 0.35037220843672456
fake_alarms: 0.6061039580352885
missed_alarms: 0.09718670076726342
  
Exp: /0_0_15 Sample: p_18_20 Run: 4 th: 4.0
accuracy: 0.4943955164131305
precision: 0.20362903225806453
recall: 0.7952755905511811
f1: 0.32423756019261635
fake_alarms: 0.5597543693906472
missed_alarms: 0.2047244094488189
  
(Dif Sample) Exp: /0_0_15 Sample: p_18_20 Run: 4 th: 4.0
accuracy: 0.525723

(Dif Sample) Exp: /0_0_15 Sample: p_18_20 Run: 6 th: 4.0
accuracy: 0.3991157556270096
precision: 0.2044967880085653
recall: 0.9769820971867008
f1: 0.3382027445772466
fake_alarms: 0.7086313781592751
missed_alarms: 0.023017902813299233
  
Exp: /0_0_15 Sample: p_18_20 Run: 6 th: 4.0
accuracy: 0.39911929543634905
precision: 0.19498910675381265
recall: 0.9396325459317585
f1: 0.32295895354082094
fake_alarms: 0.6981577704298536
missed_alarms: 0.06036745406824147
  
(Dif Sample) Exp: /0_0_15 Sample: p_18_20 Run: 6 th: 4.0
accuracy: 0.41559485530546625
precision: 0.20876712328767122
recall: 0.9744245524296675
f1: 0.34386281588447654
fake_alarms: 0.6886027658559848
missed_alarms: 0.02557544757033248
  
Exp: /0_0_15 Sample: p_18_20 Run: 6 th: 4.0
accuracy: 0.49279423538831063
precision: 0.21602564102564104
recall: 0.884514435695538
f1: 0.3472436888201958
fake_alarms: 0.5777042985356636
missed_alarms: 0.11548556430446194
  
(Dif Sample) Exp: /0_0_15 Sample: p_18_20 Run: 6 th: 4.0
accuracy: 0.50643

(Dif Sample) Exp: /0_0_15 Sample: p_18_20 Run: 8 th: 4.0
accuracy: 0.3637459807073955
precision: 0.19498464687819858
recall: 0.9744245524296675
f1: 0.3249466950959488
fake_alarms: 0.7501192179303767
missed_alarms: 0.02557544757033248
  
Exp: /0_0_15 Sample: p_18_20 Run: 8 th: 4.0
accuracy: 0.32906325060048036
precision: 0.17874875868917578
recall: 0.9424083769633508
f1: 0.3005008347245409
fake_alarms: 0.781663516068053
missed_alarms: 0.05759162303664921
  
(Dif Sample) Exp: /0_0_15 Sample: p_18_20 Run: 8 th: 4.0
accuracy: 0.375
precision: 0.19624217118997914
recall: 0.9616368286445013
f1: 0.3259644560034677
fake_alarms: 0.7343824511206486
missed_alarms: 0.03836317135549872
  
Exp: /0_0_15 Sample: p_18_20 Run: 8 th: 4.0
accuracy: 0.39751801441152923
precision: 0.18995030369961347
recall: 0.900523560209424
f1: 0.3137254901960784
fake_alarms: 0.693289224952741
missed_alarms: 0.09947643979057591
  
(Dif Sample) Exp: /0_0_15 Sample: p_18_20 Run: 8 th: 4.0
accuracy: 0.44734726688102894
preci

# Testeamos Resultados sin arq paralela

In [20]:
for exp in ['/0_0_0']:
    for sample in ['p_18', 'p_18_5', 'p_18_20']:
        for th in [1.0, 1.5, 2.0, 2.5, 3.0]:
            for run in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]:
                    #---------- Load snn spikes ------------#
                    path = './patterns/sequences/massive_runs/18/'
                    Sin_out_seq = torch.load(path + '0_0_0/last_layer_results/' + sample + '/training/'+ str(run) +'/layer_5_conv.pt')                
                    Sin_out_delay = torch.load(path + '15_0_0/last_layer_results/' + sample + '/training/'+ str(run) +'/layer_5_conv.pt')                
                    combined_out = torch.cat((Sin_out_seq, Sin_out_delay), 1)

                    #--------- Train Single Neuron ------------#
                    Sin = combined_out.clone().detach()
                    T = Sin.shape[0]
                    N_out = 1
                    N_in = Sin.shape[1]
                    th = th

                    singleNeuron = STDPLIFDensePopulation(in_channels=N_in, out_channels=N_out,
                                                            weight=0.70, alpha=float(np.exp(-1e-3/10e-3)),
                                                            beta=float(np.exp(-1e-3/2e-5)), delay=0,
                                                            th=th, a_plus=.009125, a_minus=.0125625,
                                                            w_max=1.)


                    # Pre-procesamos PSpikes y NSpikes
                    dt_ltp = 20  # Cantidad de timesteps que miro hacia atras
                    dt_ltd = 70  # Cantidad de timesteps que miro hacia delante
                    PSpikes = preSpikes(T, dt_ltp, torch.zeros(T, N_in), Sin)
                    NSpikes = nextSpikes(T, dt_ltd, torch.zeros(T, N_in), Sin)

                    # Realizamos el entrenamiento STDP
                    Uprobe = np.empty([T, N_out])
                    Iprobe = np.empty([T, N_out])
                    Sprobe = np.empty([T, N_out])
    
                    n = 0
                    t = 0
                    c_l = 1.
                    while (c_l > 0.01) and (t < 300000):
                        state = singleNeuron.forward(Sin[n].unsqueeze(0), PSpikes[n], NSpikes[n-1])
                        Uprobe[n] = state.U.data.numpy()
                        Iprobe[n] = state.I.data.numpy()
                        Sprobe[n] = state.S.data.numpy()
                        n += 1
                        if n >= T:
                            n = 0
                        c_l = convergence_level(singleNeuron.fc_layer.weight[0])
                        t += 1


                    #----------- Test Single Neuron Output -------------#
                    Sin_out_seq = torch.load(path + '0_0_0/last_layer_results/' + sample + '/testing/'+ str(run) +'/layer_5_conv.pt')                                
                    Sin_out_delay = torch.load(path + '15_0_0/last_layer_results/' + sample + '/testing/'+ str(run) +'/layer_5_conv.pt')                
                    combined_out = torch.cat((Sin_out_seq, Sin_out_delay), 1)
                    
                    pat_times = np.load(path + 'pat_sequences/' + sample + '/testing/'+ str(run) +'/pat_test_' + sample + '.npy')                    
                        
                    Sin = combined_out.clone().detach()
                    T = Sin.shape[0]
                    N_out = 1
                    N_in = Sin.shape[1]

                    # Corremos la secuencia de testing sin aprendizaje
                    Uprobe = np.empty([T, N_out])
                    Iprobe = np.empty([T, N_out])
                    Sprobe = np.empty([T, N_out])
                    for n in range(T):
                        state = singleNeuron.forward_no_learning(Sin[n].unsqueeze(0))
                        Uprobe[n] = state.U.data.numpy()
                        Iprobe[n] = state.I.data.numpy()
                        Sprobe[n] = state.S.data.numpy()

                    print('Exp: ' + exp + ' Sample: '+ sample + ' Run: '  + str(run) + ' th: ' + str(th) )
                    accuracy, precision, recall, f1, fake_alarms, missed_alarms, fake_spikes_numbers = get_metrics_long_pat(T - 15, pat_times, Sprobe, 30, [])
                    print("accuracy: " + str(accuracy))
                    print("precision: " + str(precision))
                    print("recall: " + str(recall))
                    print("f1: " + str(f1))
                    print("fake_alarms: " + str(fake_alarms))
                    print("missed_alarms: " + str(missed_alarms))
                    print('  ')
            
                    #----------- Save Results -----------#
                    str_th = str(th)
                    str_th = str_th[0] + '_' + str_th[2]
                    location = path + 'parallel_results/metrics/' + sample + '/' + str(run) + '/'
                    metrics = np.array([accuracy, precision, recall, f1, fake_alarms, missed_alarms], dtype=float)
                    pathlib.Path(location).mkdir(parents=True, exist_ok=True)
                    np.save(location + 'test_metrics_' + str_th + '.npy', metrics)
            
                    #----------- Test Dif Sample Single Neuron Output -------------#
                    Sin_out_seq = torch.load(path + '0_0_0/last_layer_results/' + sample + '/dif_sample/'+ str(run) +'/layer_5_conv.pt')                                
                    Sin_out_delay = torch.load(path + '15_0_0/last_layer_results/' + sample + '/dif_sample/'+ str(run) +'/layer_5_conv.pt')                
                    combined_out = torch.cat((Sin_out_seq, Sin_out_delay), 1)
                    
                    pat_times = np.load(path + 'pat_sequences/p_18_dif_sample/pat_p_18_dif_sample.npy')                    
                        
                    Sin = combined_out.clone().detach()
                    T = Sin.shape[0]
                    N_out = 1
                    N_in = Sin.shape[1]
                    

                    # Corremos la secuencia de Dif_sample (sin restricciones) sin aprendizaje
                    Uprobe = np.empty([T, N_out])
                    Iprobe = np.empty([T, N_out])
                    Sprobe = np.empty([T, N_out])
                    for n in range(T):
                        state = singleNeuron.forward_no_learning(Sin[n].unsqueeze(0))
                        Uprobe[n] = state.U.data.numpy()
                        Iprobe[n] = state.I.data.numpy()
                        Sprobe[n] = state.S.data.numpy()

                    print('(Dif Sample) Exp: ' + exp + ' Sample: '+ sample + ' Run: '  + str(run) + ' th: ' + str(th) )
                    accuracy, precision, recall, f1, fake_alarms, missed_alarms, fake_spikes_numbers = get_metrics_long_pat(T - 15, pat_times, Sprobe, 30, [])
                    print("accuracy: " + str(accuracy))
                    print("precision: " + str(precision))
                    print("recall: " + str(recall))
                    print("f1: " + str(f1))
                    print("fake_alarms: " + str(fake_alarms))
                    print("missed_alarms: " + str(missed_alarms))
                    print('  ')
        
                    #----------- Save Results -----------#
                    location = path + 'metrics/' + sample + '/' + str(run) + '/'
                    metrics = np.array([accuracy, precision, recall, f1, fake_alarms, missed_alarms], dtype=float)
                    pathlib.Path(location).mkdir(parents=True, exist_ok=True)
                    np.save(location + 'dif_sample_metrics_' + str_th + '.npy', metrics)
                

Exp: /0_0_0 Sample: p_18 Run: 0 th: 1.0
accuracy: 0.6613290632506005
precision: 0.24300111982082867
recall: 0.5607235142118863
f1: 0.3390625
fake_alarms: 0.3202273803884415
missed_alarms: 0.4392764857881137
  
(Dif Sample) Exp: /0_0_0 Sample: p_18 Run: 0 th: 1.0
accuracy: 0.6515273311897106
precision: 0.2446351931330472
recall: 0.5831202046035806
f1: 0.34467120181405897
fake_alarms: 0.3357176919408679
missed_alarms: 0.41687979539641945
  
Exp: /0_0_0 Sample: p_18 Run: 1 th: 1.0
accuracy: 0.5276220976781425
precision: 0.21627408993576017
recall: 0.787012987012987
f1: 0.3393057110862262
fake_alarms: 0.5196403218173213
missed_alarms: 0.21298701298701297
  
(Dif Sample) Exp: /0_0_0 Sample: p_18 Run: 1 th: 1.0
accuracy: 0.5285369774919614
precision: 0.21951219512195122
recall: 0.782608695652174
f1: 0.34285714285714286
fake_alarms: 0.5188364329995231
missed_alarms: 0.21739130434782608
  
Exp: /0_0_0 Sample: p_18 Run: 2 th: 1.0
accuracy: 0.5492393915132106
precision: 0.2222222222222222
recall

KeyboardInterrupt: 

# Tomamos los promedios de los resultados

In [22]:
path = './patterns/sequences/massive_runs/18/parallel_results/'

for exp in ['/test_metrics_', '/dif_sample_metrics_']:
    for th in ['1_0', '1_5', '2_0', '2_5', '3_0', '3_5', '4_0', '4_5', '5_0', '5_5', '6_0']:
        result = 0.
        accuracy = 0.
        precision = 0.
        recall = 0.
        f1 = 0.
        fake_alarms = 0
        missed_alarms = 0
        num_results = 0
    
        for i in range(0,10):
            location = path + 'metrics/p_18/' + str(i) + '/' + str(exp) + str(th) + '.npy'
            try:
                metrics = np.load(location)
                if np.any(metrics<0):
                    print('****** NEGATIVO ******  -> ' + str(th) + ' seed: ' + str(i))
                accuracy += metrics[0]
                precision += metrics[1]
                recall += metrics[2]
                f1 += metrics[3]
                fake_alarms += metrics[4]
                missed_alarms += metrics[5]
                num_results += 1
            except OSError as e:
                print('##### Next th is missing result ' + str(i) + ' #####')
        
        
        # Print Results
        print("---------" + str(exp) + str(th) + "---------")
        #print("accuracy: " + str(round(accuracy/num_results,4)))
        print("precision: " + str(round(precision/num_results,4)))
        print("recall: " + str(round(recall/num_results,4)))
        #print("f1: " + str(round(f1/num_results,4)))
        print("fake_alarms: " + str(round(fake_alarms/num_results,4)))
        print("missed_alarms: " + str(round(missed_alarms/num_results,4)))

---------/test_metrics_1_0---------
precision: 0.2221
recall: 0.6498
fake_alarms: 0.4224
missed_alarms: 0.3502
---------/test_metrics_1_5---------
precision: 0.2242
recall: 0.5443
fake_alarms: 0.3474
missed_alarms: 0.4557
---------/test_metrics_2_0---------
precision: 0.2245
recall: 0.5271
fake_alarms: 0.3466
missed_alarms: 0.4729
---------/test_metrics_2_5---------
precision: 0.242
recall: 0.7249
fake_alarms: 0.4407
missed_alarms: 0.2751
---------/test_metrics_3_0---------
precision: 0.3581
recall: 1.0
fake_alarms: 0.4499
missed_alarms: 0.0
##### Next th is missing result 0 #####
##### Next th is missing result 1 #####
##### Next th is missing result 2 #####
##### Next th is missing result 3 #####
##### Next th is missing result 4 #####
##### Next th is missing result 5 #####
##### Next th is missing result 6 #####
##### Next th is missing result 7 #####
##### Next th is missing result 8 #####
##### Next th is missing result 9 #####
---------/test_metrics_3_5---------


ZeroDivisionError: float division by zero